# Materiais
> Neste notebook baixamos os dados de materiais da api disponibilizada pelo MGI (dados abertos) e criamos uma base.
>
> A documentação da API do MGI pode ser encontrada em https://dadosabertos.compras.gov.br/swagger-ui/index.html#/

In [1]:
import json
import ipywidgets
from fastprogress.fastprogress import master_bar, progress_bar
import pandas as pd
import requests
import os

/Users/fredguth/Code/sus/sqlmesh/.env/lib/python3.11/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


A ideia é baixar pela API todos os materiais quem possuam uma das classes de interese:

- EQUIPAMENTOS E SUPRIMENTOS DE RAIOS-X DE USO  MÉDICO,  DENTÁRIO E VETERINÁRIO
- DROGAS E MEDICAMENTOS
- MOBILIÁRIO, EQUIPAMENTOS, UTENSÍLIOS E  SUPRIMENTOS  HOSPITALARES
- MATERIAIS CIRÚRGICOS PARA CURATIVOS
- DROGAS E PRODUTOS BIOLÓGICOS DE USO VETERINÁRIO
- ALIMENTOS ESPECIAIS DIETÉTICOS E PREPARADOS ALIMENTÍCIOS
- PESTICIDAS E DESINFETANTES
- INSTRUMENTOS, EQUIPAMENTOS E SUPRIMENTOS DENTÁRIOS
- COSMÉTICOS E ARTIGOS DE TOUCADOR DE NATUREZA MEDICINAL
- JOGOS E CONJUNTOS MÉDICOS
- EQUIPAMENTOS E ARTIGOS DE LABORATÓRIO
- VESTUÁRIO HOSPITALAR E CIRÚRGICO E ITENS CORRELATOS DE  FINALIDADES ESPECIAIS
- SUBSTÂNCIAS PARA DIAGNÓSTICO "IN VITRO",  REAGENTES,  CONJUNTOS E JOGOS PARA TESTE
- INSTRUMENTOS, EQUIPAMENTOS E SUPRIMENTOS MÉDICOS  E  CIRÚRGICOS
- EQUIPAMENTOS, INSTRUMENTOS E SUPRIMENTOS OFTALMOLÓGICOS
- PRODUTOS QUÍMICOS

In [2]:
c = [6505, 6508, 6509, 6510, 6515, 6520, 6525, 6530, 6532, 6540, 6545, 6550, 6640, 6810, 6840, 8940]

In [3]:
def fetch_and_save(codes):
    base_url = 'https://dadosabertos.compras.gov.br/modulo-material/4_consultarItemMaterial'
    
    def save_json(code, page):
        response = requests.get(f'{base_url}?codigoClasse={code}&pagina={page}').json()
        with open(f'../seeds/materiais/classe={code}/page_{page}.json', 'w') as f:
            json.dump(response, f)
        return response
    
    mb = master_bar(codes)
    for code in mb:
        os.makedirs(f'../seeds/materiais/classe={code}', exist_ok=True)
        response = save_json(code, 1)
        total_pages = response['totalPaginas']
        pb = progress_bar(range(2, total_pages + 1), parent=mb, leave=False)
        for page in pb:
            save_json(code, page)

In [4]:
fetch_and_save(c)

In [14]:
def process_file(filename):
    with open(filename, 'r') as json_file:
        resultados = json.load(json_file)['resultado']
        filepath = filename.replace("page", "records")
        with open(filepath, "w") as f:
            f.write(json.dumps(resultados))

In [15]:
import fnmatch

def process_all_json_files(root_dir):
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if fnmatch.fnmatch(file, 'page_*.json'):  # Matches only files that start with 'page_' and end with '.json'
                file_path = os.path.join(root, file)
                process_file(file_path)

In [16]:

process_all_json_files('../seeds/materiais')